In [1]:
import torch
import numpy as np
import joblib
from utils.narxwithga import NARXModel
import pandas as pd

In [ ]:
import torch
import numpy as np
import joblib
from utils.narxwithga import NARXModel
import pandas as pd

# === Load scalers and model ===
input_scaler = joblib.load("scalers/input_scaler.pkl")
output_scaler = joblib.load("scalers/output_scaler.pkl")
X_test_norm = np.load("data/X_test_norm.npy")
input_dim = 24  # explicitly use 24 instead of loading from test shape

model = NARXModel(input_dim=input_dim, hidden_dim=3, output_dim=2)
model.load_state_dict(torch.load("models/best_ga_model.pth"))
model.eval()

# === Config ===
input_delay = 4
output_delay = 4
angle_dim = 4
coord_dim = 2

df = pd.read_csv("data/train_data.csv").dropna()

columns = ['0.30','0.50','0.75','0.90','0.95','f']

angles = df[['t0', 't1', 't2', 't3']].values
coords = df[['xc', 'yc']].values

# Use first `input_delay` and `output_delay` for delayed histories
delayed_inputs = angles[:input_delay]     # shape: (4, 4)
delayed_outputs = coords[:output_delay]   # shape: (4, 2)

def run_narx(input_angles):
    """
    Predicts (x, y) coordinates from a batch of joint angle predictions (Tensor).
    input_angles: Tensor of shape (batch_size, 4)
    Returns: Tensor of shape (batch_size, 2)
    """
    if input_angles.requires_grad:
        input_angles = input_angles.detach()

    input_angles = input_angles.cpu().numpy()
    
    preds = []
    for angles in input_angles:
        # Shift histories
        global delayed_inputs, delayed_outputs
        delayed_inputs = np.roll(delayed_inputs, shift=-1, axis=0)
        delayed_outputs = np.roll(delayed_outputs, shift=-1, axis=0)

        delayed_inputs[-1] = angles

        flat_outputs = delayed_outputs.flatten()   # 4 * 2 = 8
        flat_inputs = delayed_inputs.flatten()     # 4 * 4 = 16

        narx_input = np.concatenate([flat_outputs, flat_inputs])  # 8 + 16 = 24

        narx_input_scaled = input_scaler.transform([narx_input])
        input_tensor = torch.tensor(narx_input_scaled, dtype=torch.float32)

        with torch.no_grad():
            pred_scaled = model(input_tensor).detach().numpy()
            pred_original = output_scaler.inverse_transform(pred_scaled)

        delayed_outputs[-1] = pred_original[0]
        preds.append(pred_original[0])

    return torch.tensor(preds, dtype=torch.float32)

FileNotFoundError: [Errno 2] No such file or directory: 'models/input_scaler.pkl'

In [3]:
from utils.rnn import ControlRNN
import torch.nn as nn

In [ ]:
rnn_model = ControlRNN(input_size=4, hidden_size=300, num_layers=1, dropout_rate=0)
criterion = nn.MSELoss()

rnn_model.load_state_dict(torch.load("models/nn_model.pt"))

<All keys matched successfully>

In [17]:
pos_hist = torch.Tensor([[-5.652, 2.412, 6.1773],
                              [-5.652, 2.412, 6.1773],
                              [-5.652, 2.412, 6.1773],
                              [-5.652, 2.412, 6.1773]])
#torch.Tensor([[-0.77910448,  0.7426087,   1.08560483]])
teste_pos = torch.Tensor((0.1, 0.1))

((pos_hist[0][0]-teste_pos[0])**2 + (pos_hist[0][1]-teste_pos[1])**2)**0.5

tensor(6.1993)

In [71]:
torch.cat([pos_hist[1:], torch.Tensor([[1,2,3]])])

tensor([[-5.6520,  2.4120,  6.1773],
        [-5.6520,  2.4120,  6.1773],
        [-5.6520,  2.4120,  6.1773],
        [ 1.0000,  2.0000,  3.0000]])

In [22]:
def dist_pos(curr, tgt):
    return ((curr[0]-tgt[0])**2 + (curr[1]-tgt[1])**2)**0.5

In [24]:
dist = dist_pos(pos_hist[-1], teste_pos)
dist

tensor(6.1993)

In [30]:
pos_hist.shape

torch.Size([4, 3])

In [32]:
pos_hist.reshape(1,4,3)

tensor([[[-5.6520,  2.4120,  6.1773],
         [-5.6520,  2.4120,  6.1773],
         [-5.6520,  2.4120,  6.1773],
         [-5.6520,  2.4120,  6.1773]]])

In [35]:
theta_pred[0]

tensor([ 29.0643,  60.5607, 145.2716,  80.9824], grad_fn=<SelectBackward0>)

In [42]:
input_tensor = torch.tensor([0, 3, 4, 5], dtype=torch.float32)
run_narx(input_tensor)

C:\Users\pedro\AppData\Local\Temp\ipykernel_18036\113800326.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  return torch.tensor(preds, dtype=torch.float32)


tensor([[4.8368, 0.7402],
        [4.5515, 0.7548],
        [4.5053, 1.4695],
        [3.5248, 3.6261]])

In [43]:
input_tensor.shape

torch.Size([4])

In [45]:
input_tensor.reshape(1,4)

tensor([[0., 3., 4., 5.]])

In [50]:
theta_pred.reshape(4)

tensor([ 29.0643,  60.5607, 145.2716,  80.9824], grad_fn=<ViewBackward0>)

In [52]:
pos_hist

tensor([[-5.6520,  2.4120,  6.1773],
        [-5.6520,  2.4120,  6.1773],
        [-5.6520,  2.4120,  6.1773],
        [-5.6520,  2.4120,  6.1773]])

In [65]:
theta_pred = rnn_model(pos_hist.reshape(1,4,3))
pos = run_narx(theta_pred.reshape(4))
pos

tensor([[ -2.2734,  16.3800],
        [-12.2195,  38.2576],
        [-47.7669, 116.4481],
        [-27.5932,  72.0737]])

In [ ]:
dist = dist_pos(pos_hist[-1], teste_pos)

while(dist>0.005):
    theta_pred = rnn_model(pos_hist)
    pos = narx_model(theta_pred)
    #pos_hist = 
    #pos_hist = narx_model(theta_pred)